Generate Answers For Test Queries:

In [ ]:
from datasets import load_dataset
ds = load_dataset("izhx/COMP5423-25Fall-HQ-small", "default")
ds

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

validation.jsonl: 0.00B [00:00, ?B/s]

collection.jsonl:   0%|          | 0.00/87.7M [00:00<?, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

...

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating collection split:   0%|          | 0/144718 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1052 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/12000 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'text', 'answer', 'supporting_ids'],
        num_rows: 1500
    })
    collection: Dataset({
        features: ['id', 'text', 'answer', 'supporting_ids'],
        num_rows: 144718
    })
    test: Dataset({
        features: ['id', 'text', 'answer', 'supporting_ids'],
        num_rows: 1052
    })
    train: Dataset({
        features: ['id', 'text', 'answer', 'supporting_ids'],
        num_rows: 12000
    })
})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import json

input_path = "/content/drive/MyDrive/Colab Notebooks/test_prediction_dense_instruction_e5-mistral.jsonl"
                            #test_prediction_dense_instruction_e5-mistral.jsonl
data = []
with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:  # skip empty lines
            data.append(json.loads(line))

# Now `data` is a list of dictionaries (or other JSON objects)
print(len(data))  # number of records
print(data[0])    # first record

1052
{'id': '5ab3a512554299233954ff71', 'question': ' Zhang Jinghui is well known for his role as second and final Prime Minister of a puppet state located where?', 'answer': None, 'retrieved_docs': [['doc-89201', 0.8678218126296997], ['doc-113687', 0.7204535007476807], ['doc-24324', 0.7171229124069214], ['doc-58287', 0.7152056694030762], ['doc-103479', 0.7067205309867859], ['doc-124406', 0.7011304497718811], ['doc-63871', 0.6916561126708984], ['doc-82780', 0.6910148859024048], ['doc-135815', 0.6898656487464905], ['doc-141758', 0.6898007392883301]]}


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset
from tqdm import tqdm
import json

# Load Qwen model (use GPU if available)
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")



tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
def generate_answer_from_example(query, supporting_ids, collection_ds, top_k=10):
    # Fetch the actual supporting passages from collection using supporting_ids
    passages = []
    for doc_id in supporting_ids[:top_k]:  # respect top_k even if more are provided
        if doc_id in id_to_text:
            passages.append(id_to_text[doc_id])
        else:
            print(f"Warning: supporting_id {doc_id} not found in collection.")

    # Build prompt
    prompt = f"""You are a helpful QA assistant. Answer the question using only the following passages. If you can't find evidence, base your answer on your knowledge.

Question: {query}

Passages:
{'\n'.join(passages)}

Answer:"""

    print("################################################################")
    print(prompt)
    print("################################################################")

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=200, temperature=0.7)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    answer = decoded.split("Answer:")[-1].strip()

    return answer

In [ ]:
collection_ds = ds['collection']

# Optional: pre-build a lookup dict for efficiency (especially since collection is large)
id_to_text = {item['id']: item['text'] for item in collection_ds}
id_to_text

{'doc-0': 'Twitches Too. Twitches Too is a 2007 Disney Channel Original Movie. It is the sequel to the Disney Channel Original Movie "Twitches", released in 2005. The film began production in late April 2007 and was released on October 12. It aired during "Disney Channel\'s Halloween Month". The Disney Channel Original Series, "Wizards of Waverly Place", premiered following the film\'s premiere. The first trailer was released during the premiere of "High School Musical 2". On its premiere night, the movie brought in 6.96 million viewers. To date, it is the last sequel to a Disney Channel Original Movie that is not a musical.',
 'doc-1': 'The Chant of Jimmie Blacksmith (film). The Chant of Jimmie Blacksmith is a 1978 Australian drama film directed by Fred Schepisi, and starring Tom E. Lewis (billed at the time as Tommy Lewis), Freddy Reynolds and Ray Barrett. The film also featured early appearances by Bryan Brown, Arthur Dignam, and John Jarratt. It is an adaptation of the novel "The C

In [ ]:
import json
from tqdm.auto import tqdm

results = []  # to store predictions

# Wrap the iterable with tqdm for a progress bar
for i in tqdm(range(len(data)), desc="Processing validation set"):
    example = data[i]
    query = example['question']
    supporting_ids = [i[0] for i in example['retrieved_docs']]
    gold_answer = example['answer']
    example_id = example['id']

    pred_answer = generate_answer_from_example(
        query=query,
        supporting_ids=supporting_ids,
        collection_ds=collection_ds,
        top_k=10,
    )

    results.append({
        "id": example_id,
        "question": query,
        "predicted_answer": pred_answer,
        "gold_answer": gold_answer,
        "supporting_ids": supporting_ids
    })

    print(f"Predicted answer: {pred_answer}")
    print(f"Gold answer: {example['answer']}")
    print("-" * 80)

Processing validation set:   0%|          | 0/1052 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Kristin Chenoweth was probably born sometime after 1958 but before 196
Gold answer: None
--------------------------------------------------------------------------------
################################################################
You are a helpful QA assistant. Answer the question using only the following passages. If you can't find evidence, base your answer on your knowledge.

Question: El objeto Antes Llamado Disco is the seventh studio album released by Mexican band, Café Tacuba, on October 22, 2012, in this album, the band has once again collaborated with award-winning Argentine music producer, musician, and film composer?

Passages:
El Objeto Antes Llamado Disco. El objeto Antes Llamado Disco is the seventh studio album released by Mexican band, Café Tacuba, on October 22, 2012. In this album, the band has once again collaborated with award-winning Argentine music producer Gustavo Santaolalla.
Sino (Café Tacuba album). Sino 

In [ ]:
results

[{'id': '5ab3a512554299233954ff71',
  'question': ' Zhang Jinghui is well known for his role as second and final Prime Minister of a puppet state located where?',
  'predicted_answer': "Based on the passage provided, Zhang Jinghui is well-known for his role as the second and final Prime Minister of a puppet state located in Manchukuo. According to the passage, Manchukuo was a puppet state governed under a form of constitutional monarchy in northeastern China and inner Mongolia. The puppet state was ruled by a Japanese-appointed government, and Zhang Jinghui played a significant political role within this regime, serving as the country's second and final Prime Minister. Therefore, the location of the puppet state ruled by Zhang Jinghui is Manchukuo, which was established in 1932 and lasted until 1945 when it was officially abolished after World War II.",
  'gold_answer': None,
  'supporting_ids': ['doc-89201',
   'doc-113687',
   'doc-24324',
   'doc-58287',
   'doc-103479',
   'doc-124

In [ ]:
import json

output_path = "/content/drive/MyDrive/Colab Notebooks/generate_test_full.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for item in results:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"Saved {len(results)} predictions to {output_path}")

Saved 1052 predictions to /content/drive/MyDrive/Colab Notebooks/generate_test_full.jsonl
